_____________
**Noot**: Het is ten zeerste aangeraden om deze bibliotheek in de [interactieve shell van Python](https://docs.python.org/3/tutorial/interpreter.html#interactive-mode) uit te proberen om de berekeningen mee te volgen.
_____________

# 1. Eindige velden

In [1]:
from FiniteField import *
from util import *

## 1.1 Rekenen modulo $p$

We experimenteren eerst binnen het veld $\mathbb{Z}_5$

In [2]:
Z5 = IntegerField(5)
print("Z5 =", Z5)

Z5 = [0, 1, 2, 3, 4]


Je kunt gewoon itereren over het veld alsof het een lijst zou zijn.

In [3]:
size = 0
for elem in Z5:
    size += 1
    
size

5

Let wel op: de elementen van Z5 zijn niet gewoon getallen. Het zijn objecten van het type `FieldElement` die eruit zien als een getal wanneer ze afgedrukt worden. Dat kun je zien op de volgende manier:

In [4]:
one = Z5.one()
print(type(one))

<class 'FiniteField.FieldElement'>


In [5]:
isinstance(one, int)

False

Er zijn meerdere manieren om aan deze `FieldElements` zelf te geraken. Een daarvan is de volgende (Python-magie).

In [6]:
zero, one, two, three, four = Z5
print(one, three)

1 3


De meest interessante manier is wellicht met gewone indexering. Daaraan geef je de voorstelling van een element mee (wat je ziet als je het element afdrukt, dus een `int` of een `str`) en dan krijg je het overeenkomstige `FieldElement` terug. Je mag zowel een singleton als een lijst als een matrix meegeven.

In [7]:
one = Z5[1]
two, three, four = Z5[2,3,4]

print(one, two, three, four)
any([type(elem) is int for elem in (one, two, three, four)])

1 2 3 4


False

Je kunt wel met de elementen modulorekenen alsof het getallen waren. (Hier: modulo 5)

In [8]:
one + two

3

In [9]:
three * four

2

In [10]:
two / three

4

In [11]:
three ** 2

4

Typecasts gebeuren automatisch, zolang een van de termen een `FieldElem` is. Dan is het resultaat terug een `FieldElem`.

In [12]:
three + 1

4

Vermenigvuldiging en deling worden efficient uitgevoerd omdat we een generator hebben.

In [13]:
Z5.generator()

2

In [14]:
# eerst even een functie definieren om exponenten te printen
def printpow(base, exponent):
    superscript = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
    return str(base) + str(exponent).translate(superscript)

g = Z5.generator()
for i in range(len(Z5)):
    print(printpow(g,i), "=", Z5.generator_to_power(i))

2⁰ = 1
2¹ = 2
2² = 4
2³ = 3
2⁴ = 1


Of omgekeerd: de exponent opzoeken met de `generator_exponent`-methode:

In [15]:
for elem in Z5:
    if not elem.is_zero():
        print(elem, "=", printpow(g,Z5.generator_exponent(elem)))

1 = 2⁰
2 = 2¹
3 = 2³
4 = 2²


## 1.2 Veeltermen

We kunnen ook veeltermen definieren. Er zijn meerdere manieren om dat te doen:

1) Met een lijst van coefficienten (`FieldElem`s)

In [16]:
Polynomial(Z5[2,3,4,1]) 

2 + 3x + 4x² + x³

2) Met syntactische suiker

In [17]:
# x is hier een formele veelterm over Z5
# Zo weet de vertolker dat het geheel ook een veelterm over Z5 is.
x = Z5.x()
1 + x + 3 * x**2

1 + x + 3x²

De veeltermen hebben hun eigen algebra:

In [18]:
f = 1 + x**2
g = 2 * (x + 1)
print(f)
print(g)
print(f.multiply_x_to_power(5))
print(two * g)
print(f * g)
print(f // g) # deling waarbij de rest genegeerd wordt
print(f % g)

1 + x²
2 + 2x
x⁵ + x⁷
4 + 4x
2 + 2x + 2x² + 2x³
2 + 3x
2


Gehele deling is altijd gedefinieerd, maar let erop dat er een verschil is tussen "`/`" en "`//`". Niet alle veeltermen kunnen met "`/`" door elkaar gedeeld worden.

In [19]:
try:
    f / g
except ValueError as e:
    print(e)

Cannot divide these two polynomials. The remainder is non-zero.


Toepassen op een element is zoals je zou verwachten:

In [20]:
f(one)

2

Iets gevaarlijker, maar in evenzeer mogelijk:

In [21]:
f(1)

2

Nulpunten en factorisatie:

In [22]:
for root in f.find_roots():
    print(root, end=", ")

2, 3, 

In [23]:
factors = f.factor()
print(factors)
print(product(factors) == f)

[2 + x, 3 + x]
True


In [24]:
f.is_reducible()

True

Wat informatie over de coefficienten:

In [25]:
for coef in f:
    print(coef, end=", ")

1, 0, 1, 

Wat op hetzelfde neerkomt als

In [26]:
for i in range(len(f)):
    print(f[i], end=", ")

1, 0, 1, 

In [27]:
len(f), f.degree()

(3, 2)

## 1.3 Uitbreidingsvelden

We zoeken een primitieve veelterm $w(x)$ van graad 2 in $\mathbb{Z}_2[x]$ om het uitbreidingsveld $GF(2^2) = GF(4) \cong \mathbb{Z}_2[x]|_{w(x)}$ te construeren.

In [28]:
Z2 = IntegerField(2)
w = Z2.find_primitive_polynomial(2)
print("w(x) =", w)

w(x) = 1 + x + x²


Als we een nulpunt $\xi$ van deze veelterm $w(x)$ aan het veld $\mathbb{Z}_2$ zouden toevoegen, dan zouden de machten van $\xi$ de cyclische groep $<\mathbb{Z}_2[x]|_{w(x)} \setminus \{0\}, \cdot>\ \cong\ <\mathbb{Z}_2\left(\xi\right) \setminus \{0\}, \cdot>$ moeten genereren. We kijken of alle veeltermen in $\xi$ (behalve 0) van graad kleiner dan 2 inderdaad op die manier gegenereerd worden.

In [29]:
q, k = len(Z2), w.degree()
xi = Z2.x()
xi_to_power_i = 1
for i in range(q ** k - 1):
    print(printpow("ξ", i), "=", str(xi_to_power_i).replace("x", "ξ"))
    xi_to_power_i = (xi * xi_to_power_i) % w

ξ⁰ = 1
ξ¹ = ξ
ξ² = 1 + ξ


We zien dat $\xi$ inderdaad een primitief element is, want alle mogelijke veeltermen van graad 0 en graad 1 in $\xi$ (behalve de nulveelterm) zijn een macht van $\xi$.

Hiermee kunnen we dus een uitbreidingsveld $GF(2^2) = GF(4)$ construeren.

In [30]:
# Argumenten: het oorspronkelijke veld, de graad van de veelterm, de letter waarmee je
# het primitieve element wil voorstellen, en de veelterm zelf.
GF4 = ExtendedField(Z2, 2, "ξ", w)
# of alternatief: als we w nog niet hadden, kunnen we die nu laten berekenen met
# >>> GF4 = ExtendedField(Z2, 2, "ξ")
GF4

[0, 1, ξ, 1 + ξ]

Samengevat: indien je bijvoorbeeld een veld $\mathbb{Z}_2$ hebt, en dit moet uitbreiden naar $GF(4)$, met het primitief element $\xi$ waarvoor $\xi^2 = \xi + 1 \Leftrightarrow \xi^2 -\xi - 1 = 0$, dan kun je voor $w$ de veelterm $x^2 - x - 1$ definieren. Daarmee maak je dan het uitbreidingsveld `ExtendedField(Z2, 2, "ξ", w)`.

We kunnen in dit nieuwe veld opnieuw rekenen

In [31]:
xi = GF4.generator()
for i in range(len(GF4)):
    print(xi ** i)

1
ξ
1 + ξ
1


Zoals je kunt zien is de indexering zeer versatiel: je kunt zowel getallen als strings meegeven.

In [32]:
GF4[1,"ξ", "1 + ξ"]

(1, ξ, 1 + ξ)

We kunnen dit veld natuurlijk nog eens uitbreiden tot $GF(4^2) = GF(16)$.

In [33]:
ExtendedField(GF4, 2, greek_alphabet[21])

[0, 1, φ, ξ + φ, ξ + (1 + ξ)φ, 1 + φ, ξ, ξφ, 1 + ξ + ξφ, 1 + ξ + φ, ξ + ξφ, 1 + ξ, (1 + ξ)φ, 1 + (1 + ξ)φ, 1 + ξφ, 1 + ξ + (1 + ξ)φ]

## 1.4 Typecasts en promoties

Zoals eerder gezien, gebeuren er automatisch allerhande typecasts. Bijvoorbeeld wanneer je een element van $GF(2)$ optelt bij een element van $GF(4)$:

In [34]:
one_gf2 = Z2[1]
one_gf4 = GF4[1]
print("one_gf2 ∈", one_gf2.field)
print("one_gf4 ∈", one_gf4.field)
print("1 + 1   ∈", (one_gf2 + one_gf4).field)

one_gf2 ∈ [0, 1]
one_gf4 ∈ [0, 1, ξ, 1 + ξ]
1 + 1   ∈ [0, 1, ξ, 1 + ξ]


De som van een element uit een veld en een ander element uit een uitbreidingsveld wordt dus automatisch gepromoveerd tot een element van dat uitbreidingsveld, of het resultaat nu in het kleinere veld ligt of niet. Dit is vergelijkbaar met het promoveren van `int`s naar `float`s:

In [35]:
1 + 1.0

2.0

Zo kun je ook een veeterm uit $GF(2)$ toepassen op een element van $GF(4)$ of omgekeerd. 

In [36]:
x = Z2.x()
f = 1 + x + x**3
f(GF4["ξ"])

ξ

Let erop dat bepaalde syntactische suiker niet altijd werkt. Zo is het volgende onomgelijk.

In [37]:
try:
    print(f("ξ"))
except Exception:
    print("Error. Cannot compute the given expression.")

Error. Cannot compute the given expression.


Omdat, voor zover $\mathbb{Z}_2$ en $f(x)$ (gedefineerd over $\mathbb{Z}_2$) weten, het element $\xi$ helemaal niet bestaat.

Wat wel kan, is het volgende:

In [38]:
f(GF4["ξ"])

ξ

De vertolker weet immers dat `GF4["ξ"]` tot `GF4` behoort. Deze zal `f` dus eerst promoveren tot een veelterm over `GF4`, en dan op $\xi$ toepassen.

De eerste notatie zou wel gebruikt kunnen worden als $f(x)$ gedefineerd was over $GF(4)$.

In [39]:
g = f.cast_to_field(GF4)
g("ξ")

ξ

Als je zeker wil zijn dat je het juiste element aanspreekt (zoals in het voorbeeld hierboven), gebruik je best de notatie `GF4["ξ"]` om het element $\xi$ voor te stellen. Je kunt $\xi$ natuurlijk ook opslaan in een variabele:
`xi = GF4["ξ"]`. Syntactische suiker is dan ook volledig optioneel.

Als je niet meer zeker bent over het type, dan kun je altijd `type` en/of `field` gebruiken.

In [40]:
xi = GF4["ξ"]
print("type  =", type(xi), "\nfield =", xi.field)

type  = <class 'FiniteField.FieldElement'> 
field = [0, 1, ξ, 1 + ξ]


Over het algemeen werken de meeste notaties die je zou verwachten. Werken die niet, dan gebruik je `FieldElem`s zoals hierboven aangeven.

## 1.5 Factorisatie van $x^n - 1$

We vertrekken met de volgende veelterm in $GF(4)[x]$. Merk op dat aagezien $-1 = 1 \text{ (mod 2)}$, deze veelterm ook uitgedrukt kan worden als $x^n + 1$

In [41]:
n = 5
x = GF4.x()
p = x ** n - 1
p

1 + x⁵

We hebben nood aan een uitbreidingsveld $GF(4^k)$ waarin we een element $\beta$ kunnen vinden dat een primitieve $n$-de wortel uit 1 is. $k$ is hierbij het kleinste getal waarvoor $4^k = 1 \text{ (mod } n \text{)}$

In [42]:
beta = GF4.nth_root_of_unity(n)
GF4k = beta.field
k = GF4k.divisor.degree() # de graad van de veelterm die gebruikt is om GF(4^k) te construeren
print("β =",beta, "\nk =", k)

β = ξ + (1 + ξ)α 
k = 2


We zien dat $\beta$ niet in $GF(4)$ gelegen is, maar in $GF(4^k)$. Volgens de theorie zijn alle machten van $\beta$ een oplossing van $p$.

In [43]:
all([p(beta ** i).is_zero() for i in range(n+1)])

True

We bepalen de factorisatie met behulp van de cyclotomische nevenklassen modulo $n$ over $GF(4)$.

In [44]:
cyclotomic_cosets(4, n)

[[0], [1, 4], [2, 3]]

Als we deze machten van $\beta$ combineren, hebben we een factorisatie van $x^n - 1$ over $GF(4)$.

In [45]:
factors = GF4.factor_nth_root(n)
for factor in factors:
    print(factor)

1 + x
1 + (1 + ξ)x + x²
1 + ξx + x²


En inderdaad:

In [46]:
product(factors) == p

True

# 2. Codetheorie

## 2.1 Lineaire algebra

In [2]:
from LinearAlgebra import *

Vectoren en matrices zijn vanzelfsprekend zodra je in eindige velden kunt werken.

In [48]:
Z3 = IntegerField(3)

In [49]:
v = Vector(Z3[1,2,0,2,0])
w = Vector(Z3[0,2,0,0,1])
v + w

[1, 1, 0, 2, 1]

In [50]:
2 * w

[0, 1, 0, 0, 2]

Indexatie en iteratie:

In [51]:
for i in range(len(v)):
    print(v[i], end=", ")

1, 2, 0, 2, 0, 

In [52]:
for component in v:
    print(component, end=", ")

1, 2, 0, 2, 0, 

In [53]:
v.get_elems()

[1, 2, 0, 2, 0]

De matrices zijn ook van de partij:

In [54]:
A = Matrix(Z3[[1,2],[2,0]])
print(A * A.transpose())

[2, 2]
[2, 1]



In [55]:
for row in A:
    # elke rij is gewoon een lijst, geen vector
    print(row)

[1, 2]
[2, 0]


Vectoren zijn hier rijmatrices, geen kolommatrices. Dat is van belang voor de vermenigvuldiging.

In [56]:
v = Vector(Z3[0,1])
try:
    print(A * v)
except Exception:
    print("Je moest de volgorde omkeren om te vermenigvuldigen. v.A =", v * A)

Je moest de volgorde omkeren om te vermenigvuldigen. v.A = [2, 0]


Verder nog enkele courante bewerkingen in verband met stelsels:

In [57]:
A = Matrix(Z3[[1,1,2],[1,0,2],[1,2,1]])
A.echelon_form()
# De andere variant A.to_echelon_form() is in-place en wijzigt A zonder iets terug te geven

[1, 1, 2]
[0, 2, 0]
[0, 0, 2]

In [58]:
A.is_singular()

False

De elementaire rijoperaties

In [59]:
print(A,"\nR_0 <-> R_1")
A.rowswap(0,1)
print(A, "\nR_0 -> 2*R_0")
A.rowmul(0, Z3[2])
print(A, "\nR0 -> R_0 + 2*R_1")
A.add_multiple_of_row(0, 1, 2)
print(A)

[1, 1, 2]
[1, 0, 2]
[1, 2, 1]
 
R_0 <-> R_1
[1, 0, 2]
[1, 1, 2]
[1, 2, 1]
 
R_0 -> 2*R_0
[2, 0, 1]
[1, 1, 2]
[1, 2, 1]
 
R0 -> R_0 + 2*R_1
[1, 2, 2]
[1, 1, 2]
[1, 2, 1]



Het stelsel $Ax^T = b^T$ met $x$ en $b$ rijvectoren.

In [60]:
b = Vector(Z3[1,2,0])
x = A.solve(b)
x

[1, 2, 1]

In [61]:
X = Matrix.from_vector(x).transpose()
B = Matrix.from_vector(b).transpose()

A * X == B

True

Deze matrix $H$ is natuurlijk niet uniek: door elementaire rijoperaties toe te passen heb je een nieuwe matrix die ook een oplossing zou kunnen zijn.

## 2.2 Lineaire blokcodes

In [6]:
from Code import *

Beschouw een (8,3)-code $\mathbb{C}$ over $GF(3)$, met een generatormatrix $G$.

In [63]:
GF3 = IntegerField(3)
G = Matrix(GF3[[2,0,1,1,2,1,0,0],[0,2,0,1,1,2,1,0],[0,0,2,0,1,1,2,1]])
G

[2, 0, 1, 1, 2, 1, 0, 0]
[0, 2, 0, 1, 1, 2, 1, 0]
[0, 0, 2, 0, 1, 1, 2, 1]

Hiermee kunnen we een informatiewoord $i \in GF(3)^3$ encoderen.

In [64]:
i = Vector(GF3[1,0,2])
c = i * G
c

[2, 0, 2, 1, 1, 0, 1, 2]

We zoeken nu een pariteitstestmatrix $H$ die voldoet aan $GH^T = 0$, m.a.w. $\forall c: c = iG \Rightarrow cH^T = 0$, m.a.w. de rijen van $H$ zijn basis voor de nulruimte van $G$. Die matrix $H$ kun je vinden met de methode `nullspace`.

In [65]:
H = G.nullspace()
print("H =")
print(H)

print((G*H.transpose()).is_zero())

H =
[1, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 1, 1]
[2, 0, 1, 0, 0, 1, 1, 1]
[2, 1, 2, 0, 1, 1, 1, 1]
[0, 2, 2, 1, 1, 1, 1, 1]

True


Deze is echter niet uniek: als we elementaire rijoperaties toepassen op H, blijft deze nog steeds de kern van $G$ opspannen. Dat komt omdat de rijruimte van een matrix onveranderd blijft bij het nemen van lineaire combinaties van rijen.

In [66]:
H.rowswap(1,2)
H.add_multiple_of_row(0,1, GF3[2])
(G*H.transpose()).is_zero()

True

Volgens de theorie is de $H$ eenvoudig te berekenen als $G$ van de vorm $[I | P]$ is. Dan is $H = [-P^T | I]$. Om dat te doen kunnen we dus eerst een equivalente code voor $\mathbb{C}$ berekenen.

In [67]:
G_ = G.reduced_echelon_form()
G_

[1, 0, 0, 2, 0, 1, 1, 2]
[0, 1, 0, 2, 2, 1, 2, 0]
[0, 0, 1, 0, 2, 2, 1, 2]

In [68]:
H_ = G_.nullspace_from_standard_form()
H_

[1, 1, 0, 1, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0, 0]
[2, 2, 1, 0, 0, 1, 0, 0]
[2, 1, 2, 0, 0, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 0, 1]

En opnieuw

In [69]:
G_ * H_.transpose()

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]

De afstand van de code is het grootste getal $d$ zodat elke verzameling van $d$ kolommen van $H$ lineair onafhankelijk is.

In [70]:
d = H.transpose().num_independent_rows()
t = max_num_corrected_errors(d)
d, t

(5, 2)

Voor enkele van de voorgaande bewerkingen hadden we ook een `LinearBlockCode`-object kunnen maken.

In [71]:
code = LinearBlockCode(G, H) # De H is optioneel. In dat geval wordt H nu berekend.
i = Vector(GF3[1,0,2])
c = code.encode(i)
c

[2, 0, 2, 1, 1, 0, 1, 2]

Deze code heeft een decoderingstabel

In [72]:
table = code.decode_table()

# Dit is enkel om af te drukken
print("syndroom        |   fout")
print(42 * "=")
i = 0
N = 10
for syndrome in table:
    print(syndrome, "|", table[syndrome])
    i += 1
    if i >= N:
        break
print("...\n(nog", len(table) - N, " rijen)")

syndroom        |   fout
[0, 0, 0, 0, 0] | [0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 0, 2, 0] | [1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0] | [2, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 2] | [0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 2, 2, 1] | [0, 2, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 2, 2] | [0, 0, 1, 0, 0, 0, 0, 0]
[0, 2, 0, 1, 1] | [0, 0, 2, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1] | [0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 2] | [0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 1, 1] | [0, 0, 0, 0, 1, 0, 0, 0]
...
(nog 119  rijen)


Die kunnen we uittesten. Brengen we nu een fout aan die die code kan verbeteren:

In [73]:
e = Vector(GF3[0,0,2,0,0,0,0,1])
v = c + e
print("v     =",v, "\nwt(e) =", v.distance(c))

v     = [2, 0, 1, 1, 1, 0, 1, 0] 
wt(e) = 2


De ontvanger van dit woord kan de fout terug opzoeken in de syndroomtabel, en zo het oorspronkelijke codewoord $v - e$ reconstrueren:

In [74]:
s = v * code.Ht # Ht = H^T
e = table[s]
e

[0, 0, 2, 0, 0, 0, 0, 1]

Of wat ook kan:

In [75]:
code.find_error(v)

[0, 0, 2, 0, 0, 0, 0, 1]

## 2.3 Cyclische codes

We construeren een generatorveelterm over $GF(3)$

In [88]:
GF3 = IntegerField(3)
x = GF3.x()
n = 8
factors = GF3.factor_nth_root(n)
print("x^n - 1 = ", end="")
for factor in factors:
    print("(" + str(factor) + ")", end="")

x^n - 1 = (2 + x)(2 + x + x²)(1 + x²)(1 + x)(2 + 2x + x²)

Neem bijvoorbeeld $g(x) = (2 + x + x^2)(1 + x^2)(1 + x)$ van graad 5. Dat geeft dus een (8,3)-code over $GF(3)$.

In [77]:
g = (2 + x + x**2)*(1 + x**2)*(1 + x)
k = n - g.degree()
g

2 + x² + x³ + 2x⁴ + x⁵

De afstand van deze code is $min_{c\ \ne\ 0} wt(c) = min_{i\ \ne\ 0} wt(i \cdot g(x))$

In [78]:
from itertools import product as cartesian_product
d = float("inf")
for i_coefs in cartesian_product(GF3, repeat=k): # i_coefs zit in GF(3) x GF(3) x GF(3)
    i = Polynomial(i_coefs)
    if not i.is_zero():
        c = g * i
        c_coefs = Vector(c.coef)
        d = min(d, c_coefs.weight())
d

5

Het aantal fouten dat de code kan decoderen, is dus

In [79]:
max_num_corrected_errors(d)

2

We kunnen weer een informatiewoord van lengte $k$ coderen en over het kanaal zenden.

In [80]:
i = Polynomial(GF3[1,2,1])
c = i * g
c

2 + x + 2x⁴ + x⁶ + x⁷

In [81]:
e = Polynomial(GF3[0,1,0,0,2,0,0,0])
v = c + e
v

2 + 2x + x⁴ + x⁶ + x⁷

Opnieuw is er een klasse van cyclische codes die een aantal processen automatiseert en een syndroomtabel opstelt.

In [82]:
code = CyclicCode(g, n)
code.encode(i)

2 + x + 2x⁴ + x⁶ + x⁷

In [83]:
table = code.decode_table()

# Dit is enkel om af te drukken
print("syndroom\nfout")
print(20 * "=")
i = 0
N = 4
for syndrome in table:
    if i >= len(table) - N:
        print("s(x) =", syndrome, "\ne(x) =", table[syndrome], "\n", 20*"-")
    i += 1
print("...\n(nog", len(table) - N, " rijen)")

syndroom
fout
s(x) = 1 + 2x + x⁴ 
e(x) = x⁶ + x⁷ 
 --------------------
s(x) = 1 + x² + 2x³ + 2x⁴ 
e(x) = x⁶ + 2x⁷ 
 --------------------
s(x) = 2 + 2x² + x³ + x⁴ 
e(x) = 2x⁶ + x⁷ 
 --------------------
s(x) = 2 + x + 2x⁴ 
e(x) = 2x⁶ + 2x⁷ 
 --------------------
...
(nog 125  rijen)


Met deze syndroomtabel kunnen we het oorsprokelijke codewoord terugvinden. Ter herinnering, dit was het ontvangen woord $v(x)$.

In [84]:
v

2 + 2x + x⁴ + x⁶ + x⁷

In [85]:
s = v % g
e = table[s]
c = v - e
i = c / g
i

1 + 2x + x²

Wat inderdaad het geencodeerde informatiewoord was.

Het kan ook in 1 stap:

In [86]:
code.decode(v)

1 + 2x + x²

## 2.4 BCH-codes

Ook hiervoor is er gewoon een klasse. We gebruiken als voorbeeld een code over $GF(11)$. Eerst maken we een uitbreidingsveld van $GF(11)$ en een primitieve $n$-de wortel uit om de code te definieren (zie pagina 30 in de cursustekst).

In [7]:
(n,q,t,l) = (15,11,3,2)
GFq = IntegerField(q)
x = GFq.x()
w = 7 + x + x**2
GF121 = ExtendedField(GFq, 2, "α", w)
alpha = GF121["α"]

# β zoeken
k = 1
while q**k % n != 1:
    k += 1
beta = alpha ** ((q**k - 1)//n)

Nu kunnen we een code maken.

In [8]:
code = BCHCode(n, GFq, t, l, beta)

De generator heeft als nulpunten alle machten van $\beta$ gaande van $l$ tot $2t$, alsook alle machten die daarmee in de cyclotomische nevenklassen zitten.

In [9]:
code.g

3 + 10x + 8x² + 4x⁴ + 6x⁶ + 6x⁷ + x⁸

We kunnen een veelterm $v(x)$ decoderen met met PGZ (de `verbose=True/False` is optioneel)

In [10]:
v = Polynomial(GFq[3, 5, 4, 3, 6, 9, 10, 9, 8, 10, 8, 4, 6, 4, 7])

In [11]:
Lambda = code.pgz(v, verbose=True)
e = code.forney(v, Lambda, verbose=True)
c = v - e
i = c / code.g
print("i(x) =", i)

PGZ on v(x) = 3 + 5x + 4x² + 3x³ + 6x⁴ + 9x⁵ + 10x⁶ + 9x⁷ + 8x⁸ + 10x⁹ + 8x¹⁰ + 4x¹¹ + 6x¹² + 4x¹³ + 7x¹⁴
Trying ν = 3. H is singular.
Trying ν = 2. H is nonsingular.
Result: Λ(x) = 1 + 4x + x² 

Forney's algorithm on v(x) = 3 + 5x + 4x² + 3x³ + 6x⁴ + 9x⁵ + 10x⁶ + 9x⁷ + 8x⁸ + 10x⁹ + 8x¹⁰ + 4x¹¹ + 6x¹² + 4x¹³ + 7x¹⁴
X_k:  [3, 4] = ['β^6', 'β^9']
S(x) = 1 + 8x + 3x³ + 10x⁴ + x⁵
Ω(x) = 1 + x
Λ'(x) = 4 + 2x
Y_k: =  [3, 8]
e(x) =  3x⁶ + 8x⁹ 

i(x) = 1 + 2x + 3x² + 4x³ + 5x⁴ + 6x⁵ + 7x⁶


Of met BM:

In [16]:
Lambda = code.bm(v, verbose=True)
e = code.forney(v, Lambda, verbose=True)
i = code.correct_error(v, e, verbose=True)

Berlekamp-Massey on v(x) = 3 + 5x + 4x² + 3x³ + 6x⁴ + 9x⁵ + 10x⁶ + 9x⁷ + 8x⁸ + 10x⁹ + 8x¹⁰ + 4x¹¹ + 6x¹² + 4x¹³ + 7x¹⁴ 

n+d |  Δ |  n |  d |        Λ(x) |  Λ*(x) | 
 == | == | == | == |          == |     == | 
  0 |  / |  0 |  0 |           1 |      0 | 
  1 |  1 |  0 |  1 |           x |      1 | 
  2 | 10 |  1 |  1 |       1 + x |      1 | 
  3 |  2 |  1 |  2 |  9 + x + x² | 6 + 6x | 
  4 |  5 |  2 |  2 | 1 + 4x + x² | 6 + 6x | 
  5 |  0 |  3 |  2 | 1 + 4x + x² | 6 + 6x | 
  6 |  0 |  4 |  2 | 1 + 4x + x² | 6 + 6x | 

Result: Λ(x) =  1 + 4x + x² 

Forney's algorithm on v(x) = 3 + 5x + 4x² + 3x³ + 6x⁴ + 9x⁵ + 10x⁶ + 9x⁷ + 8x⁸ + 10x⁹ + 8x¹⁰ + 4x¹¹ + 6x¹² + 4x¹³ + 7x¹⁴
X_k:  [3, 4] = ['β^6', 'β^9']
S(x) = 1 + 8x + 3x³ + 10x⁴ + x⁵
Ω(x) = 1 + x
Λ'(x) = 4 + 2x
Y_k: =  [3, 8]
e(x) =  3x⁶ + 8x⁹ 

Decoding:
c(x) = v(x) - e(x) =  3 + 5x + 4x² + 3x³ + 6x⁴ + 9x⁵ + 7x⁶ + 9x⁷ + 8x⁸ + 2x⁹ + 8x¹⁰ + 4x¹¹ + 6x¹² + 4x¹³ + 7x¹⁴
i(x) = c(x) / g(x) =  1 + 2x + 3x² + 4x³ + 5x⁴ + 6x⁵ + 7x⁶ 



of alles tegelijk

In [15]:
print("i(x) =", code.decode(v, verbose=False))     # BM
print("i(x) =", code.decode_pgz(v, verbose=False)) # PGZ

i(x) = 1 + 2x + 3x² + 4x³ + 5x⁴ + 6x⁵ + 7x⁶
i(x) = 1 + 2x + 3x² + 4x³ + 5x⁴ + 6x⁵ + 7x⁶
